# Federated Learning for Text Generation

This tutorial builds on the MNIST one, and uses the Shakespeare data set. It does not require discussing any new concepts as they've all already been introduced.